In [1]:
# import of standard Python libraries for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# import needed objects from Scikit learn library for machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

In [3]:
# I would like to see all rows and columns of dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# reading dataset
df = pd.read_csv("final_data.csv")

In [5]:
df.head()

,elevation,massif_num,lon,lat,temp_soil_0.005_m,temp_soil_0.08_m,liquid_water_in_soil,frozen_water_in_soil,risk_index,aval_type,whiteness_albedo,net_radiation,drainage,runoff,snow_melting_rate,rainfall_rate,surface_temperature,surface_snow_amount,thickness_of_snowfall,snow_thickness_1D,snow_thickness_3D,snow_thickness_5D,snow_thickness_7D,snow_water_1D,snow_water_3D,snow_water_5D,snow_water_7D,penetration_ram_resistance,thickness_of_wet_snow_top_of_snowpack,thickness_of_frozen_snow_top_of_snowpack,acccidental_risk_index,day,surface_air_pressure_mean,near_surface_humidity_mean,relative_humidity_mean,freezing_level_altitude_mean,rain_snow_transition_altitude_mean,air_temp_max,wind_speed_max,snowfall_rate_max,nebulosity_max,air_temp_min,massif_name,aval_accident,aval_event
0,1500.0,1,6.64493,46.17685,277.709158,279.014864,0.323705,0.0,6.0,6.0,0.2,3.251180,0.000018,0.000019,0.0,0.000096,277.709158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010-10-01,84550.445,0.005956,80.023400,2268.572667,422.333333,283.38647,4.92501,0.0,0.988523,277.14102,Chablais,0,0
1,1500.0,1,6.64493,46.17685,279.496314,280.143477,0.310928,0.0,6.0,6.0,0.2,79.345799,0.000018,0.000000,0.0,0.000000,279.496314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010-10-02,84550.445,0.006067,58.228333,1080.672643,40.151042,288.89374,4.92201,0.0,0.758338,281.01437,Chablais,0,0
2,1500.0,1,6.64493,46.17685,278.575394,280.282568,0.305990,0.0,6.0,6.0,0.2,44.215499,0.000018,0.000000,0.0,0.000000,278.575394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010-10-03,84550.445,0.006655,61.123905,1194.040409,11.500000,290.59950,3.32201,0.0,0.627164,282.84122,Chablais,0,0
3,1500.0,1,6.64493,46.17685,280.511023,281.446828,0.301774,0.0,6.0,6.0,0.2,73.435071,0.000018,0.000000,0.0,0.000000,280.511023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010-10-04,84550.445,0.005970,58.574722,1937.796566,718.493056,291.77880,5.22301,0.0,0.996815,281.35240,Chablais,0,0
4,1500.0,1,6.64493,46.17685,280.790508,281.635283,0.319800,0.0,6.0,6.0,0.2,17.299822,0.000018,0.000008,0.0,0.000047,280.790508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010-10-05,84550.445,0.007777,94.231740,1837.712598,515.784722,284.17500,4.12301,0.0,0.982423,280.60666,Chablais,0,0


In [6]:
# viewing all variables to decide which are not useful
df.columns

Index(['elevation', 'massif_num', 'lon', 'lat', 'temp_soil_0.005_m',
       'temp_soil_0.08_m', 'liquid_water_in_soil', 'frozen_water_in_soil',
       'risk_index', 'aval_type', 'whiteness_albedo', 'net_radiation',
       'drainage', 'runoff', 'snow_melting_rate', 'rainfall_rate',
       'surface_temperature', 'surface_snow_amount', 'thickness_of_snowfall',
       'snow_thickness_1D', 'snow_thickness_3D', 'snow_thickness_5D',
       'snow_thickness_7D', 'snow_water_1D', 'snow_water_3D', 'snow_water_5D',
       'snow_water_7D', 'penetration_ram_resistance',
       'thickness_of_wet_snow_top_of_snowpack',
       'thickness_of_frozen_snow_top_of_snowpack', 'acccidental_risk_index',
       'day', 'surface_air_pressure_mean', 'near_surface_humidity_mean',
       'relative_humidity_mean', 'freezing_level_altitude_mean',
       'rain_snow_transition_altitude_mean', 'air_temp_max', 'wind_speed_max',
       'snowfall_rate_max', 'nebulosity_max', 'air_temp_min', 'massif_name',
       'aval_accid

In [7]:
# removing variables not needed for random forest model
df_clean = df.drop(columns=["massif_num","lon","lat","aval_type", "acccidental_risk_index"])

In [8]:
# viewing if data type of the variables is suitable for Random Forest
# there are 2 string categorical variables that needs to be transformed: day and massif name
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540818 entries, 0 to 540817
Data columns (total 40 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   elevation                                 540818 non-null  float64
 1   temp_soil_0.005_m                         540818 non-null  float64
 2   temp_soil_0.08_m                          540818 non-null  float64
 3   liquid_water_in_soil                      540818 non-null  float64
 4   frozen_water_in_soil                      540818 non-null  float64
 5   risk_index                                540818 non-null  float64
 6   whiteness_albedo                          540818 non-null  float64
 7   net_radiation                             540818 non-null  float64
 8   drainage                                  540818 non-null  float64
 9   runoff                                    540818 non-null  float64
 10  snow_melting_rate   

In [9]:
df_clean.shape

(540818, 40)

In [10]:
# for transformation of massif names I will use OneHotEncoding with pd.get_dummies method
# selecting of values for dummies
df_clean.massif_name.unique()

array(['Chablais', 'Aravis', 'Mont-Blanc', 'Bauges', 'Beaufortin',
       'Hte-tarent', 'Chartreuse', 'Belledonne', 'Maurienne', 'Vanoise',
       'Hte-maurie', 'Gdes-rouss', 'Thabor', 'Vercors', 'Oisans',
       'Pelvoux', 'Queyras', 'Devoluy', 'Champsaur', 'Parpaillon',
       'Ubaye', 'Ht_Var-Ver', 'Mercantour'], dtype=object)

In [11]:
massifs = ('Chablais', 'Aravis', 'Mont-Blanc', 'Bauges', 'Beaufortin',
       'Hte-tarent', 'Chartreuse', 'Belledonne', 'Maurienne', 'Vanoise',
       'Hte-maurie', 'Gdes-rouss', 'Thabor', 'Vercors', 'Oisans',
       'Pelvoux', 'Queyras', 'Devoluy', 'Champsaur', 'Parpaillon',
       'Ubaye', 'Ht_Var-Ver', 'Mercantour')

In [12]:
# creating initial dataframe
df_massifs = pd.DataFrame(massifs, columns=['massif_name'])

# generate binary values using get_dummies
dum_df = pd.get_dummies(df_massifs, columns=["massif_name"], prefix="massif")

# merge initial dataframe with dummies
df_massifs = df_massifs.join(dum_df)

# merge final datatset with dataframe with dummies
df_clean = df_clean.merge(df_massifs, how="left", on="massif_name")

In [13]:
# checking dataset with dummy variables
df_clean.head()

,elevation,temp_soil_0.005_m,temp_soil_0.08_m,liquid_water_in_soil,frozen_water_in_soil,risk_index,whiteness_albedo,net_radiation,drainage,runoff,snow_melting_rate,rainfall_rate,surface_temperature,surface_snow_amount,thickness_of_snowfall,snow_thickness_1D,snow_thickness_3D,snow_thickness_5D,snow_thickness_7D,snow_water_1D,snow_water_3D,snow_water_5D,snow_water_7D,penetration_ram_resistance,thickness_of_wet_snow_top_of_snowpack,thickness_of_frozen_snow_top_of_snowpack,day,surface_air_pressure_mean,near_surface_humidity_mean,relative_humidity_mean,freezing_level_altitude_mean,rain_snow_transition_altitude_mean,air_temp_max,wind_speed_max,snowfall_rate_max,nebulosity_max,air_temp_min,massif_name,aval_accident,aval_event,massif_Aravis,massif_Bauges,massif_Beaufortin,massif_Belledonne,massif_Chablais,massif_Champsaur,massif_Chartreuse,massif_Devoluy,massif_Gdes-rouss,massif_Ht_Var-Ver,massif_Hte-maurie,massif_Hte-tarent,massif_Maurienne,massif_Mercantour,massif_Mont-Blanc,massif_Oisans,massif_Parpaillon,massif_Pelvoux,massif_Queyras,massif_Thabor,massif_Ubaye,massif_Vanoise,massif_Vercors
0,1500.0,277.709158,279.014864,0.323705,0.0,6.0,0.2,3.251180,0.000018,0.000019,0.0,0.000096,277.709158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,84550.445,0.005956,80.023400,2268.572667,422.333333,283.38647,4.92501,0.0,0.988523,277.14102,Chablais,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1500.0,279.496314,280.143477,0.310928,0.0,6.0,0.2,79.345799,0.000018,0.000000,0.0,0.000000,279.496314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-02,84550.445,0.006067,58.228333,1080.672643,40.151042,288.89374,4.92201,0.0,0.758338,281.01437,Chablais,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1500.0,278.575394,280.282568,0.305990,0.0,6.0,0.2,44.215499,0.000018,0.000000,0.0,0.000000,278.575394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-03,84550.445,0.006655,61.123905,1194.040409,11.500000,290.59950,3.32201,0.0,0.627164,282.84122,Chablais,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1500.0,280.511023,281.446828,0.301774,0.0,6.0,0.2,73.435071,0.000018,0.000000,0.0,0.000000,280.511023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-04,84550.445,0.005970,58.574722,1937.796566,718.493056,291.77880,5.22301,0.0,0.996815,281.35240,Chablais,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1500.0,280.790508,281.635283,0.319800,0.0,6.0,0.2,17.299822,0.000018,0.000008,0.0,0.000047,280.790508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-05,84550.445,0.007777,94.231740,1837.712598,515.784722,284.17500,4.12301,0.0,0.982423,280.60666,Chablais,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# getting rid of redundant variable
df_clean = df_clean.drop(columns=['massif_name'])

In [15]:
# second categorical variable to transform is day
# slicing is used to get years and months from day variable
# and after we need to transform years and months to integers
df_clean["year"] = (df_clean.day.str[:4]).astype(int)
df_clean["month"] = (df_clean.day.str[5:7]).astype(int)
df_clean.head()

,elevation,temp_soil_0.005_m,temp_soil_0.08_m,liquid_water_in_soil,frozen_water_in_soil,risk_index,whiteness_albedo,net_radiation,drainage,runoff,snow_melting_rate,rainfall_rate,surface_temperature,surface_snow_amount,thickness_of_snowfall,snow_thickness_1D,snow_thickness_3D,snow_thickness_5D,snow_thickness_7D,snow_water_1D,snow_water_3D,snow_water_5D,snow_water_7D,penetration_ram_resistance,thickness_of_wet_snow_top_of_snowpack,thickness_of_frozen_snow_top_of_snowpack,day,surface_air_pressure_mean,near_surface_humidity_mean,relative_humidity_mean,freezing_level_altitude_mean,rain_snow_transition_altitude_mean,air_temp_max,wind_speed_max,snowfall_rate_max,nebulosity_max,air_temp_min,aval_accident,aval_event,massif_Aravis,massif_Bauges,massif_Beaufortin,massif_Belledonne,massif_Chablais,massif_Champsaur,massif_Chartreuse,massif_Devoluy,massif_Gdes-rouss,massif_Ht_Var-Ver,massif_Hte-maurie,massif_Hte-tarent,massif_Maurienne,massif_Mercantour,massif_Mont-Blanc,massif_Oisans,massif_Parpaillon,massif_Pelvoux,massif_Queyras,massif_Thabor,massif_Ubaye,massif_Vanoise,massif_Vercors,year,month
0,1500.0,277.709158,279.014864,0.323705,0.0,6.0,0.2,3.251180,0.000018,0.000019,0.0,0.000096,277.709158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,84550.445,0.005956,80.023400,2268.572667,422.333333,283.38647,4.92501,0.0,0.988523,277.14102,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10
1,1500.0,279.496314,280.143477,0.310928,0.0,6.0,0.2,79.345799,0.000018,0.000000,0.0,0.000000,279.496314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-02,84550.445,0.006067,58.228333,1080.672643,40.151042,288.89374,4.92201,0.0,0.758338,281.01437,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10
2,1500.0,278.575394,280.282568,0.305990,0.0,6.0,0.2,44.215499,0.000018,0.000000,0.0,0.000000,278.575394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-03,84550.445,0.006655,61.123905,1194.040409,11.500000,290.59950,3.32201,0.0,0.627164,282.84122,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10
3,1500.0,280.511023,281.446828,0.301774,0.0,6.0,0.2,73.435071,0.000018,0.000000,0.0,0.000000,280.511023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-04,84550.445,0.005970,58.574722,1937.796566,718.493056,291.77880,5.22301,0.0,0.996815,281.35240,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10
4,1500.0,280.790508,281.635283,0.319800,0.0,6.0,0.2,17.299822,0.000018,0.000008,0.0,0.000047,280.790508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010-10-05,84550.445,0.007777,94.231740,1837.712598,515.784722,284.17500,4.12301,0.0,0.982423,280.60666,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10


In [16]:
# other way how to transform day is to count number od days since day 0
# first step of this transformation is change from string to datetime
df_clean['day'] = pd.to_datetime(df_clean['day'], format='%Y-%m-%d')

In [17]:
# setting day 0 and calculation of following days
fromDate = min(df_clean['day'])
df_clean['timedelta'] = ((df_clean['day']) - fromDate).dt.days.astype(int)
print(df_clean[['day', 'timedelta']].head())
df_clean.drop('day', axis = 1, inplace = True)

         day  timedelta
0 2010-10-01          0
1 2010-10-02          1
2 2010-10-03          2
3 2010-10-04          3
4 2010-10-05          4


In [18]:
# checking dataset with new time variables
df_clean.head()

,elevation,temp_soil_0.005_m,temp_soil_0.08_m,liquid_water_in_soil,frozen_water_in_soil,risk_index,whiteness_albedo,net_radiation,drainage,runoff,snow_melting_rate,rainfall_rate,surface_temperature,surface_snow_amount,thickness_of_snowfall,snow_thickness_1D,snow_thickness_3D,snow_thickness_5D,snow_thickness_7D,snow_water_1D,snow_water_3D,snow_water_5D,snow_water_7D,penetration_ram_resistance,thickness_of_wet_snow_top_of_snowpack,thickness_of_frozen_snow_top_of_snowpack,surface_air_pressure_mean,near_surface_humidity_mean,relative_humidity_mean,freezing_level_altitude_mean,rain_snow_transition_altitude_mean,air_temp_max,wind_speed_max,snowfall_rate_max,nebulosity_max,air_temp_min,aval_accident,aval_event,massif_Aravis,massif_Bauges,massif_Beaufortin,massif_Belledonne,massif_Chablais,massif_Champsaur,massif_Chartreuse,massif_Devoluy,massif_Gdes-rouss,massif_Ht_Var-Ver,massif_Hte-maurie,massif_Hte-tarent,massif_Maurienne,massif_Mercantour,massif_Mont-Blanc,massif_Oisans,massif_Parpaillon,massif_Pelvoux,massif_Queyras,massif_Thabor,massif_Ubaye,massif_Vanoise,massif_Vercors,year,month,timedelta
0,1500.0,277.709158,279.014864,0.323705,0.0,6.0,0.2,3.251180,0.000018,0.000019,0.0,0.000096,277.709158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84550.445,0.005956,80.023400,2268.572667,422.333333,283.38647,4.92501,0.0,0.988523,277.14102,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10,0
1,1500.0,279.496314,280.143477,0.310928,0.0,6.0,0.2,79.345799,0.000018,0.000000,0.0,0.000000,279.496314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84550.445,0.006067,58.228333,1080.672643,40.151042,288.89374,4.92201,0.0,0.758338,281.01437,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10,1
2,1500.0,278.575394,280.282568,0.305990,0.0,6.0,0.2,44.215499,0.000018,0.000000,0.0,0.000000,278.575394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84550.445,0.006655,61.123905,1194.040409,11.500000,290.59950,3.32201,0.0,0.627164,282.84122,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10,2
3,1500.0,280.511023,281.446828,0.301774,0.0,6.0,0.2,73.435071,0.000018,0.000000,0.0,0.000000,280.511023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84550.445,0.005970,58.574722,1937.796566,718.493056,291.77880,5.22301,0.0,0.996815,281.35240,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10,3
4,1500.0,280.790508,281.635283,0.319800,0.0,6.0,0.2,17.299822,0.000018,0.000008,0.0,0.000047,280.790508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84550.445,0.007777,94.231740,1837.712598,515.784722,284.17500,4.12301,0.0,0.982423,280.60666,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2010,10,4


In [19]:
# verifying there are no null values in dataset
(df_clean.apply(lambda x: x.isnull().sum())).sum()

0

In [20]:
# Labels are the values we want to predict
labels_ev = np.array(df_clean['aval_event'])
labels_ac = np.array(df_clean['aval_accident'])

# Remove the labels from the features
features_ev = df_clean.drop(columns=['aval_event', 'aval_accident'])
features_ac = df_clean.drop(columns=['aval_event', 'aval_accident'])

# Saving feature names for later use
feature_list_ev = list(features_ev.columns)
feature_list_ac = list(features_ac.columns)

# Convert to numpy array
features_ev = np.array(features_ev)
features_ac = np.array(features_ac)

In [21]:
train_features_ac, test_features_ac, train_labels_ac, test_labels_ac = train_test_split(features_ac, labels_ac, test_size = 0.33, random_state = 42)

In [22]:
train_features_ev, test_features_ev, train_labels_ev, test_labels_ev = train_test_split(features_ev, labels_ev, test_size = 0.33, random_state = 42)

In [23]:
print('Training Features Shape:', train_features_ac.shape)
print('Training Labels Shape:', train_labels_ac.shape)
print('Testing Features Shape:', test_features_ac.shape)
print('Testing Labels Shape:', test_labels_ac.shape)

Training Features Shape: (362348, 62)
Training Labels Shape: (362348,)
Testing Features Shape: (178470, 62)
Testing Labels Shape: (178470,)


In [24]:
print('Training Features Shape:', train_features_ev.shape)
print('Training Labels Shape:', train_labels_ev.shape)
print('Testing Features Shape:', test_features_ev.shape)
print('Testing Labels Shape:', test_labels_ev.shape)

Training Features Shape: (362348, 62)
Training Labels Shape: (362348,)
Testing Features Shape: (178470, 62)
Testing Labels Shape: (178470,)


In [25]:
# https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652

In [26]:
rfc_ac=RandomForestClassifier()

In [27]:
%%time
rfc_ac.fit(train_features_ac,train_labels_ac)

label_pred_ac=rfc_ac.predict(test_features_ac)

Wall time: 3min 18s


In [28]:
print("Accuracy:",metrics.accuracy_score(test_labels_ac, label_pred_ac))

Accuracy: 0.9959657085224407


In [29]:
%%time
rfc_cv_score_ac = cross_val_score(rfc_ac, features_ac, labels_ac, cv=10, scoring="roc_auc")

Wall time: 38min 40s


In [30]:
print(confusion_matrix(test_labels_ac, label_pred_ac))

[[177170      0]
 [   720    580]]


In [31]:
print(classification_report(test_labels_ac, label_pred_ac))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    177170
           1       1.00      0.45      0.62      1300

    accuracy                           1.00    178470
   macro avg       1.00      0.72      0.81    178470
weighted avg       1.00      1.00      1.00    178470



In [32]:
rfc_cv_score_ac, rfc_cv_score_ac.mean()

(array([0.76127969, 0.78424816, 0.8648653 , 0.52097924, 0.24656857,
        0.27135597, 0.17323505, 0.06238934, 0.33373028, 0.735082  ]),
 0.4753733599829152)

In [33]:
rfc_ev=RandomForestClassifier()

In [34]:
%%time
rfc_ev.fit(train_features_ev,train_labels_ev)

label_pred_ev=rfc_ev.predict(test_features_ev)

Wall time: 3min 15s


In [35]:
print("Accuracy:",metrics.accuracy_score(test_labels_ev, label_pred_ev))

Accuracy: 0.9981901720177061


In [36]:
%%time
rfc_cv_score_ev = cross_val_score(rfc_ev, features_ev, labels_ev, cv=10, scoring="roc_auc")

Wall time: 37min 11s


In [37]:
confusion_matrix(test_labels_ev, label_pred_ev)

array([[177760,      0],
       [   323,    387]], dtype=int64)

In [38]:
print(classification_report(test_labels_ev, label_pred_ev))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    177760
           1       1.00      0.55      0.71       710

    accuracy                           1.00    178470
   macro avg       1.00      0.77      0.85    178470
weighted avg       1.00      1.00      1.00    178470



In [39]:
print(rfc_cv_score_ev)

print("Mean AUC Score - Random Forest: ", rfc_cv_score_ev.mean())

[0.59758812 0.50783215 0.87489601 0.60829673 0.28648612 0.21437767
 0.30898971 0.17467018 0.3234928  0.62295317]
Mean AUC Score - Random Forest:  0.45195826616785684


In [41]:
features_ev_df = df_clean.drop(columns=['aval_event', 'aval_accident'])
features_ac_df = df_clean.drop(columns=['aval_event', 'aval_accident'])

In [43]:
pd.DataFrame({'Variable':features_ac_df.columns,
              'Importance':rfc_ac.feature_importances_}).sort_values('Importance', ascending=False)

,Variable,Importance
61,timedelta,0.043925
29,freezing_level_altitude_mean,0.043190
34,nebulosity_max,0.043004
30,rain_snow_transition_altitude_mean,0.037618
22,snow_water_7D,0.033756
32,wind_speed_max,0.032537
6,whiteness_albedo,0.031323
18,snow_thickness_7D,0.031218
21,snow_water_5D,0.031195
20,snow_water_3D,0.031122


In [44]:
pd.DataFrame({'Variable':features_ev_df.columns,
              'Importance':rfc_ev.feature_importances_}).sort_values('Importance', ascending=False)

,Variable,Importance
61,timedelta,0.050318
29,freezing_level_altitude_mean,0.044588
34,nebulosity_max,0.042980
22,snow_water_7D,0.035888
30,rain_snow_transition_altitude_mean,0.035567
21,snow_water_5D,0.033773
20,snow_water_3D,0.032752
18,snow_thickness_7D,0.031726
32,wind_speed_max,0.031529
7,net_radiation,0.029818


In [ ]:
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
